# IWBDA 2023 SBOL 3 Tutorial

### September 2023
This tutorial code goes with the slides at:

https://github.com/SynBioDex/Community-Media/blob/master/2023/IWBDA23/SBOL3-IWBDA-2023.pptx

Install the modules. SBOL-utilities also installs pySBOL3 and tyto as dependencies.

In [1]:
!pip install -q sbol_utilities

Import the modules

In [2]:
from sbol3 import *
from sbol_utilities.calculate_sequences import compute_sequence
from sbol_utilities.component import *
import tyto

Set the default namespace for new objects and create a document

In [3]:
set_namespace('https://synbiohub.org/public/igem/')
doc = Document()

# Slide 26: GFP expression cassette
Construct a simple part and add it to the Document.

In [4]:
i13504 = Component('i13504', SBO_DNA)
i13504.name = 'iGEM 2016 interlab reporter'
i13504.description = 'GFP expression cassette used for 2016 iGEM interlab study'
i13504.roles.append(tyto.SO.engineered_region)

Add the GFP expression cassette to the document.
Notice that the object added is also returned, so this can be used as a pass-through call.

In [ ]:
doc.add(i13504)

# Slide 28: Expression cassette parts
Here we will create a part-subpart hierarchy. We will also start using (SBOL-Utilities)[https://github.com/synbiodex/sbol-utilities] to make it easier to create parts and to assemble those parts into a hierarchy.

First, create the RBS component...

In [6]:
b0034, b0034_seq = doc.add(rbs('B0034', sequence='aaagaggagaaa', name='RBS (Elowitz 1999)'))

Next, create the GFP component

In [7]:
e0040_sequence = 'atgcgtaaaggagaagaacttttcactggagttgtcccaattcttgttgaattagatggtgatgttaatgggcacaaattttctgtcagtggagagggtgaaggtgatgcaacatacggaaaacttacccttaaatttatttgcactactggaaaactacctgttccatggccaacacttgtcactactttcggttatggtgttcaatgctttgcgagatacccagatcatatgaaacagcatgactttttcaagagtgccatgcccgaaggttatgtacaggaaagaactatatttttcaaagatgacgggaactacaagacacgtgctgaagtcaagtttgaaggtgatacccttgttaatagaatcgagttaaaaggtattgattttaaagaagatggaaacattcttggacacaaattggaatacaactataactcacacaatgtatacatcatggcagacaaacaaaagaatggaatcaaagttaacttcaaaattagacacaacattgaagatggaagcgttcaactagcagaccattatcaacaaaatactccaattggcgatggccctgtccttttaccagacaaccattacctgtccacacaatctgccctttcgaaagatcccaacgaaaagagagaccacatggtccttcttgagtttgtaacagctgctgggattacacatggcatggatgaactatacaaataataa'
e0040, _ = doc.add(cds('E0040', sequence=e0040_sequence, name='GFP'))

Finally, create the terminator

In [8]:
b0015_sequence = 'ccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacgctctctactagagtcacactggctcaccttcgggtgggcctttctgcgtttata'
b0015, _ = doc.add(terminator('B0015', sequence=b0015_sequence, name='double terminator'))

Now construct the part-subpart hierarchy and order the parts: RBS before CDS, CDS before terminator

In [ ]:
order(b0034, e0040, i13504)
order(e0040, b0015, i13504)

# Slide 30: Location of a SubComponent

Here we add base coordinates to SubComponents.

But first, use `compute_sequence` to get the full sequence for the BBa_I13504 device

See http://parts.igem.org/Part:BBa_I13504

In [10]:
i13504_seq = compute_sequence(i13504)

`compute_sequence` added `Range`s to the subcomponents. Check one of those ranges to see that the values are what we expect.

The expected range of the terminator is (733, 861).

In [ ]:
b0015_subcomponent = next(f for f in i13504.features if f.instance_of == b0015.identity)
b0015_range = b0015_subcomponent.locations[0]
print(f'Range of {b0015.display_name}: ({b0015_range.start}, {b0015_range.end})')

# Slide 32: GFP production from expression cassette
In this example, we will create a system representation that includes DNA, proteins, and interactions.

First, create the system representation.  `functional_component` creates this for us.

In [ ]:
i13504_system = functional_component('i13504_system')
doc.add(i13504_system)

The system has two physical subcomponents, the expression construct and the expressed GFP protein. We already created the expression construct. Now create the GFP protein.
`ed_protein` creates an "externally defined protein"

In [13]:
gfp = add_feature(i13504_system, ed_protein('https://www.fpbase.org/protein/gfpmut3/', name='GFP'))

Now create the part-subpart hierarchy.

In [14]:
i13504_subcomponent = add_feature(i13504_system, i13504)

Use a ComponentReference to link SubComponents in a multi-level hierarchy

In [15]:
e0040_subcomponent = next(f for f in i13504.features if f.instance_of == e0040.identity)
e0040_reference = ComponentReference(i13504_subcomponent, e0040_subcomponent)
i13504_system.features.append(e0040_reference)

Make the Interaction.

Interaction type: SBO:0000589 (genetic production)

Participation roles: SBO:0000645 (template), SBO:0000011 (product)

In [ ]:
add_interaction(tyto.SBO.genetic_production,
                participants={gfp: tyto.SBO.product, e0040_reference: tyto.SBO.template})

# Slide 34: Concatenating and Reusing Components
Connecting the `i13504_system` with promoters to drive expression is much like building `i13504`: selecting features and ordering them.

First, we create the two promoters:

In [17]:
J23101_sequence = 'tttacagctagctcagtcctaggtattatgctagc'
J23101, _ = doc.add(promoter('J23101', sequence=J23101_sequence))
J23106_sequence = 'tttacggctagctcagtcctaggtatagtgctagc'
J23106, _ = doc.add(promoter('J23106', sequence=J23106_sequence))

Then we connect them to `ComponentReference` objects that reference the i13504 `SubComponent`

In [ ]:
device1 = doc.add(functional_component('interlab16device1'))
device1_i13504_system = add_feature(device1, SubComponent(i13504_system))
order(J23101, ComponentReference(device1_i13504_system, i13504_subcomponent), device1)
device2 = doc.add(functional_component('interlab16device2'))
device2_i13504_system = add_feature(device2, SubComponent(i13504_system))
order(J23106, ComponentReference(device2_i13504_system, i13504_subcomponent), device2)
print(f'Device 1 second subcomponent points to {device1.constraints[0].object.lookup().refers_to.lookup().instance_of}')

# Slide 35: Making a collection
We will just add the two devices that we built here, not all five on the slide

In [ ]:
interlab16 = doc.add(Collection('interlab16',members=[device1, device2]))
print(f'Members are {", ".join(m.lookup().display_id for m in interlab16.members)}')

# Slide 36: Creating strains
Describing an engineered strain is much like the other components we have defined, just with different types.

First, we create `Component` objects for the DH5-a E. coli strain and the backbone vector we will use for the transfection.

In [20]:
ecoli = doc.add(strain('Ecoli_DH5_alpha'))
pSB1C3 = doc.add(Component('pSB1C3', SBO_DNA, roles=[tyto.SO.plasmid_vector]))

Now create the engineered strain

In [21]:
device1_ecoli = doc.add(strain('device1_ecoli'))

Create a local description of the vector as the combination of Device 1 and pSB1C3

In [ ]:
plasmid = LocalSubComponent(SBO_DNA, roles=[tyto.SO.plasmid_vector], name="Interlab Device 1 in pSB1C3")
device1_ecoli.features.append(plasmid)
device1_subcomponent = contains(plasmid, device1)
contains(plasmid, pSB1C3)
order(device1, pSB1C3, device1_ecoli)

And put the vector into the transformed strain

In [ ]:
contains(ecoli, plasmid, device1_ecoli)

# Slide 37: Defining an abstract interface
To refer to the GFP, we need to peer down two levels of hierarchy

In [24]:
gfp_in_i13504_system = add_feature(device1_ecoli, ComponentReference(in_child_of=device1_i13504_system, refers_to=gfp))
gfp_in_strain = add_feature(device1_ecoli, ComponentReference(in_child_of=device1_subcomponent, refers_to=gfp_in_i13504_system))
device1_ecoli.interface = Interface(outputs=[gfp_in_strain])

# Slide 38: Linking to a model

In [29]:
ode_model = doc.add(Model('my_iBioSIM_ODE', 'https://synbiohub...', tyto.EDAM.SBML, tyto.SBO.continuous_framework))
device1_ecoli.models.append(ode_model)

## Validate the document
`Document.validate` returns a validation report. If the report is empty, the document is valid.

In [ ]:
report = doc.validate()
if report:
    print('Document is not valid')
    print(f'Document has {len(report.errors)} errors')
    print(f'Document has {len(report.warnings)} warnings')
else:
    print('Document is valid')

# Finally, write the data out to a file

In [26]:
doc.write('i13504.nt', file_format=SORTED_NTRIPLES)